In [4]:
import pandas as pd
import numpy as np
import math

from sklearn.ensemble import RandomForestRegressor

# We add Dash necessary librairies
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

X = pd.read_csv('X.csv')
y = pd.read_csv('y.csv')
y = np.ravel(y)

# Load prices/barrio
prices = np.load('price_barrio.npy',allow_pickle='TRUE').item()

X['sq_mt_built'] = X['sq_mt_built']
X['buy_price/barrio'] = X['buy_price/barrio']

# We train a simple RF model

model = RandomForestRegressor(max_depth = 10, n_estimators = 150)
model.fit(X, y)

# We create a DataFrame to store the features' importance and their corresponding label
df_feature_importances = pd.DataFrame(model.feature_importances_*100,columns=["Importance"],index=X.columns)
df_feature_importances = df_feature_importances.sort_values("Importance", ascending=False)


# We record the name, min, mean and max of the three most important features
dropdown_1_label = df_feature_importances.index[0]
dropdown_1_min = math.floor(X[dropdown_1_label].min())
dropdown_1_mean = round(X[dropdown_1_label].mean())
dropdown_1_max = round(X[dropdown_1_label].max())

# dropdown_2_label = df_feature_importances.index[1]
# dropdown_2_min = math.floor(X[dropdown_2_label].min())
# dropdown_2_mean = round(X[dropdown_2_label].mean())
# dropdown_2_max = round(X[dropdown_2_label].max())

dropdown_3_label = df_feature_importances.index[2]
dropdown_3_min = math.floor(X[dropdown_3_label].min())
dropdown_3_mean = round(X[dropdown_3_label].mean())
dropdown_3_max = round(X[dropdown_3_label].max())


###############################################################################

app = dash.Dash(external_stylesheets=[dbc.themes.CYBORG])

# We apply basic HTML formatting to the layout
app.layout = html.Div(style={'textAlign': 'center', 'width': '700px', 'font-family': 'Verdana'},

                    children=[

                        # The same logic is applied to the following names / sliders
                        html.H3(children="SIMULATION TOOL"),


                        # We display the most important feature's name
                        html.H5(children="Built square meters"),

                        # The Dash Slider is built according to Feature #1 ranges
                        dcc.Slider(
                            id='X1_slider',
                            min=0,
                            max=500,
                            step=50,
                            value=dropdown_1_mean,
                           marks={i: '{}'.format(i) for i in range(0,500,50)},),

                        html.Br(),

                        # The same logic is applied to the following names / sliders
                        html.H5(children="Buy price/neighborhood"),

                        dcc.Dropdown(
                            id='X2_slider',
                            options=[{'label': clave, 'value': valor} for clave, valor in prices.items()],
                            placeholder="Select a neighborhood, or type the name",
                            value= None
                        ),

                        html.Br(),

                        html.H6(id="price_barrio"),

                        html.Br(),

                        html.H5(children="Number of rooms"),

                        dcc.Slider(
                            id='X3_slider',
                            min=0,
                            max=5,
                            step=1,
                            value=dropdown_3_mean,
                            marks={i: '{}'.format(i) for i in range(0,6)},
                        ),

                        # The predicting result will be displayed and updated here
                        html.H4(id="prediction_result"),

                    ])

@app.callback(Output(component_id="price_barrio",component_property="children"),
              [Input("X2_slider","value")])

def output_price(price):
    return u' has a built square metre price of: {}€'.format(price)

# The callback function will provide one "Ouput" in the form of a string (=children)
@app.callback(Output(component_id="prediction_result",component_property="children"),
# The values correspnding to the three sliders are obtained by calling their id and value property
              [Input("X1_slider","value"), Input("X2_slider","value"), Input("X3_slider","value")])

# The input variable are set in the same order as the callback Inputs
def update_prediction(X1, X2, X3):

    # We create a NumPy array in the form of the original features
    # Except for the X1, X2 and X3, all other non-influencing parameters are set to their mean
    input_X = np.array([X3,
                        X1,
                        X2,
                        X["District_no"].mean(),
                        X["is_renewal_needed"].mean(),
                        X["has_parking"].mean(),
                        X["type"].mean(),
                       ]).reshape(1,-1)

    # Prediction is calculated based on the input_X array
    prediction = int(model.predict(input_X)[0])

    # And retuned to the Output of the callback function
    return "Prediction: {:,}€".format(prediction).replace(',', ' ')

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
